<a href="https://colab.research.google.com/github/KashyapCKotak/AI-ML-experiments/blob/master/SeeInDark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



print('Dowloading Sony subset... (25GB)')
download_file_from_google_drive('10kpAcvldtcb9G2ze5hTcF1odzu4V_Zvh', './Sony.zip')

#print('Dowloading Fuji subset... (52GB)')
#download_file_from_google_drive('12hvKCjwuilKTZPe9EZ7ZTb-azOmUA3HT', 'dataset/Fuji.zip')

os.system('unzip ./Sony.zip -d dataset')
#os.system('unzip dataset/Fuji.zip -d dataset')


Dowloading Sony subset... (25GB)


In [0]:
shutil.rmtree('dataset', ignore_errors=True, onerror=None)

In [0]:
os.system('unzip ./Sony.zip -d dataset')
#os.system('unzip dataset/Fuji.zip -d dataset')

0

**Main Train Code:**
=================================

In [3]:
!pip install rawpy

     |████████████████████████████████| 686kB 2.8MB/s 


In [13]:
from __future__ import division
import os, time, scipy.io
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import rawpy
import glob
import keras

Using TensorFlow backend.


In [0]:
input_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/short'
gt_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/long'
os.mkdir('result_Sony')
checkpoint_dir = './result_Sony/'
result_dir = './result_Sony/'

In [0]:
train_fns = glob.glob(gt_dir + '0*.ARW')
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

In [0]:
ps = 512  # patch size for training
save_freq = 500

In [0]:
DEBUG = 0
if DEBUG == 1:
    save_freq = 2
    train_ids = train_ids[0:5]

In [0]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)

In [0]:
def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output

In [0]:
def network(input):
    
    #definition: slim.conv2d(inputs,number of outputs, kernel shape)
    #definition: slim.max_pool2d(inputs,kernel_size, padding)
    #conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1=keras.layers.Conv2D(input_shape=[],filters=32,kernel_size=(3,3),strides=(1,1))(inputs)
    conv1=keras.layers.LeakyReLU(alpha=0.2)(conv1)
    #conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    conv1=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1))(conv1)
    conv1=keras.layers.LeakyReLU(alpha=0.2)(conv1)
    #pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')
    pool1=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same")(conv1)
    
    #conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1))(pool1)
    conv2=keras.layers.LeakyReLU(alpha=0.2)(conv2)
    #conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    conv2=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1))(conv2)
    conv2=keras.layers.LeakyReLU(alpha=0.2)(conv2)
    #pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')
    pool2=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same")(conv2)
    
    #conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1))(pool2)
    conv3=keras.layers.LeakyReLU(alpha=0.2)(conv3)
    #conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    conv3=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1))(conv3)
    conv3=keras.layers.LeakyReLU(alpha=0.2)(conv3)
    #pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')
    pool3=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same")(conv3)
    
    #conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1))(pool3)
    conv4=keras.layers.LeakyReLU(alpha=0.2)(conv4)
    #conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    conv4=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1))(conv4)
    conv4=model.add(keras.layers.LeakyReLU(alpha=0.2)(conv4)
    #pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')
    pool4=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same"))(conv4)
    
    #conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5=keras.layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1))(pool4)
    conv5=keras.layers.LeakyReLU(alpha=0.2)(conv5)
    #conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')
    conv5=keras.layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1))(conv5)
    conv5=keras.layers.LeakyReLU(alpha=0.2)(conv5)
    
    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    #conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1))(up6)
    conv6=keras.layers.LeakyReLU(alpha=0.2)(conv6)
    #conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')
    conv6=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1))(conv6)
    conv6=keras.layers.LeakyReLU(alpha=0.2)(conv6)
    
    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    #conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1))(up7)
    conv7=keras.layers.LeakyReLU(alpha=0.2)(conv7)
    #conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')
    conv7=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1))(conv7)
    conv7=keras.layers.LeakyReLU(alpha=0.2)(conv7)
    
    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    #conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1))(up8)
    conv8=keras.layers.LeakyReLU(alpha=0.2)(conv8)
    #conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')
    conv8=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1))(conv8)
    conv8=keras.layers.LeakyReLU(alpha=0.2)(conv8)
    
    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    #conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1))(up9)
    conv9=keras.layers.LeakyReLU(alpha=0.2)(conv9)
    #conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')
    conv9=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1))(conv9)
    conv9=keras.layers.LeakyReLU(alpha=0.2)(conv9)
    
    #conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    conv9=keras.layers.Conv2D(filters=12,kernel_size=(1,1),strides=(1,1))(conv9)
    #no activation function for this last layer
    
    out = tf.depth_to_space(conv10, 2)
    return out